In [3]:
%pip install plotly --user
%pip install pandas==0.24.1 --user
%pip install tensorboardX --user
%pip install bs4 --user
%pip install -U auquan_toolbox --user

     |████████████████████████████████| 7.2MB 3.0MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-cp36-none-any.whl size=9533 sha256=ab92039daf5eabd568dfb6cc25e9ec5e453dc53422bb93b5227a77bf5901c3bf
  Stored in directory: /home/vishal/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
  Using cached https://files.pythonhosted.org/packages/64/ce/f1f7f1a53b4c9

     |████████████████████████████████| 92kB 1.1MB/s eta 0:00:01
     |████████████████████████████████| 112kB 2.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/02/e5/38518af393f7c214357079ce67a317307936896e961e35450b70fad2a9cf/rsa-4.0-py2.py3-none-any.whl
  Created wheel for auquan-toolbox: filename=auquan_toolbox-2.1.91-cp36-none-any.whl size=101185 sha256=3267c173717cce478a91345fcd76367ee6b9231b4e301455c37c81b16913e2a6
  Stored in directory: /home/vishal/.cache/pip/wheels/e2/79/d5/528b025f1ccb10266884d2edbda98ed61af3b89ac0485b4f69
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=c2e501c8c33009face2030bd6c7cf512c29c94eeed3ba0261de07eb56d07e945
  Stored in directory: /home/vishal/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
  Created wheel for absl-py: filename=absl_py-0.9.0-cp36-none-any.whl size=119399 sha256=a60649c4de79064ffdcb126805fee011d60bb95fd070a32a61aa55d62fa23bee
  Stored in di

In [1]:
from backtester.trading_system_parameters import TradingSystemParameters
from backtester.features.feature import Feature
from backtester.dataSource.yahoo_data_source import YahooStockDataSource
from backtester.timeRule.custom_time_rule import CustomTimeRule
from backtester.executionSystem.simple_execution_system import SimpleExecutionSystem
from backtester.orderPlacer.backtesting_order_placer import BacktestingOrderPlacer
from backtester.trading_system import TradingSystem
from backtester.constants import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta

In [2]:
class MyTradingParams(TradingSystemParameters):
    
    def __init__(self, tradingFunctions):
        self.__tradingFunctions = tradingFunctions
        super(MyTradingParams, self).__init__()
        self.__dataSetId = 'equity_data'
        self.__instrumentIds = self.__tradingFunctions.getSymbolsToTrade()
        self.__startDate = '2015/01/02'
        self.__endDate = '2017/07/31'
        
    def getDataParser(self):
        return YahooStockDataSource(
            cachedFolderName = 'historicalData/',
            dataSetId = self.__dataSetId,
            instrumentIds = self.__instrumentIds,
            startDateStr = self.__startDate,
            endDateStr = self.__endDate
        )
    
    def getTimeRuleForUpdates(self):
        return CustomTimeRule(
            startDate = self.__startDate, 
            endDate = self.__endDate, 
            frequency = 'D', 
            sample = '30'
        )
    
    def getFrequencyOfFeatureUpdates(self):
        return timedelta(days = 1)
    
    def getStartingCapital(self):
        if len(self.__tradingFunctions.getSymbolsToTrade()) > 0:
            return 1000*len(self.__tradingFunctions.getSymbolsToTrade())
        else:
            return 30000
        
    def getCustomFeatures(self):
        return {
            'my_custom_feature': MyCustomFeature,
            'prediction': TrainingPredictionFeature,
            'zero_fees': FeesCalculator,
            'benchmark_PnL': BuyHoldPnL,
            'score': ScoreFeature
        }
    
    def getInstrumentFeatureConfigDicts(self):
        predictionDict = {
            'featureKey': 'prediction',
            'featureId': 'prediction',
            'params': {}
        }
        feesConfigDict = {
            'featureKey': 'fees',
            'featureId': 'zero_fees',
            'params': {}
        }
        profitlossConfigDict = {
            'featureKey': 'pnl',
            'featureId': 'pnl',
            'params': {
                'price': self.getPriceFeatureKey(),
                'fees': 'fees'
            }
        }
        capitalConfigDict = {
            'featureKey': 'capital',
            'featureId': 'capital',
            'params': {
                'price': 'adjClose',
                'fees': 'fees',
                'capitalReqPercent': 0.95
            }
        }
        benchmarkDict = {
            'featureKey': 'benchmark',
            'featureId': 'benchmark_PnL',
            'params': {'pnlKey': 'pnl'}
        }
        scoreDict = {
            'featureKey': 'score',
            'featureId': 'score',
            'params': {
                'featureName1': 'pnl',
                'featureName2':'benchmark'
            }
        }

        stockFeatureConfigs = self.__tradingFunctions.getInstrumentFeatureConfigDicts()
        
        return {
            INSTRUMENT_TYPE_STOCK: stockFeatureConfigs + [
                predictionDict,
                feesConfigDict,
                profitlossConfigDict,
                capitalConfigDict,
                benchmarkDict, 
                scoreDict
            ]
        }
    
    def getMarketFeatureConfigDicts(self):
        scoreDict = {
            'featureKey': 'score',
            'featureId': 'score_ll',
            'params': {
                'featureName': self.getPriceFeatureKey(),
                'instrument_score_feature': 'score'
            }
        }
        
        return [scoreDict]
    
    def getPrediction(self, time, updateNum, instrumentManager):
        predictions = pd.Series(index = self.__instrumentIds)
        predictions = self.__tradingFunctions.getPrediction(time, updateNum, instrumentManager, predictions)
        
        return predictions
    
    def getExecutionSystem(self):
        return SimpleExecutionSystem(
            enter_threshold = 0.7,
            exit_threshold = 0.55,
            longLimit = 1,
            shortLimit = 1,
            capitalUsageLimit = 0.10*self.getStartingCapital(),
            enterlotSize = 1, 
            exitlotSize = 1,
            limitType = 'L', 
            price = 'adjClose'
        )
    
    def getOrderPlacer(self):
        return BacktestingOrderPlacer()

    def getLookbackSize(self):
        return 120
    
    def getPriceFeatureKey(self):
        return 'adjClose'
    
    def getInstrumentsIds(self):
        return self.__instrumentIds
                

In [3]:
class TrainingPredictionFeature(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        tf = MyTradingFunctions()
        t = MyTradingParams(tf)
        
        return t.getPrediction(time, updateNum, instrumentManager)
    
class FeesCalculator(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        
        return pd.Series(0, index = instrumentManager.getAllInstrumentsByInstrumentId())
    
class BuyHoldPnL(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        instrumentLookbackData = instrumentManager.getLookbackInstrumentFeatures()

        priceData = instrumentLookbackData.getFeatureDf('adjClose')
         
        if len(priceData) < 2:
            return pd.Series(0, index = instrumentManager.getAllInstrumentsByInstrumentId())
        else:
            bhpnl = instrumentLookbackData.getFeatureDf(featureKey).iloc[-1]
            bhpnl += priceData.iloc[-1] - priceData.iloc[-2]

        return bhpnl
    
class ScoreFeature(Feature):
    
    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        instrumentLookbackData = instrumentManager.getLookbackInstrumentFeatures()
        if len(instrumentLookbackData.getFeatureDf(featureParams['featureName1'])) > 0:
            feature1 = instrumentLookbackData.getFeatureDf(featureParams['featureName1']).iloc[-1]
            feature2 = instrumentLookbackData.getFeatureDf(featureParams['featureName2']).iloc[-1]
            
            for instrumentId in feature1.index:
                pnls = instrumentLookbackData.getFeatureDf('pnl')[instrumentId]
                positions = instrumentLookbackData.getFeatureDf('position')[instrumentId]
                
                print(instrumentId)
                print('pnl: %.2f'%pnls[-1])
                if len(positions) > 2 and np.abs(positions[-1] - positions[-2]) > 0:
                    print('Position changed to: %.2f'%positions[-1])
            
            toRtn = (feature1 - feature2) / feature2.abs()
            toRtn[toRtn.isnull()] = 0
            toRtn[toRtn == np.Inf] = 0
        else:
            toRtn=0

        return toRtn


In [4]:
class MyTradingFunctions():

    def __init__(self):
        self.count = 0
        self.params = {}

    def getSymbolsToTrade(self):
        return ['AAPL']
    
    def getInstrumentFeatureConfigDicts(self):
        ma1Dict = {
            'featureKey': 'ma_90',
            'featureId': 'moving_average',
            'params': {
                'period': 90,
                'featureName': 'adjClose'
            }
        }
        mom30Dict = {
            'featureKey': 'mom_30',
            'featureId': 'momentum',
            'params': {
                'period': 30,
                'featureName': 'adjClose'
            }
        }
        mom10Dict = {
            'featureKey': 'mom_10',
            'featureId': 'momentum',
            'params': {
                'period': 10,
                'featureName': 'adjClose'
            }
        }
        
        return [ma1Dict, mom10Dict, mom30Dict]
    
    def getPrediction(self, time, updateNum, instrumentManager, predictions):
        
        lookbackInstrumentFeatures = instrumentManager.getLookbackInstrumentFeatures()
        
        def hurst_f(input_ts, lags_to_test=20):
            tau = []
            lagvec = []
            for lag in range(2, lags_to_test):
                pp = np.subtract(input_ts[lag:], input_ts[:-lag]) 
                lagvec.append(lag) 
                tau.append(np.sqrt(np.std(pp)))  
            m = np.polyfit(np.log10(lagvec), np.log10(tau), 1)
            hurst = m[0]*2
            return hurst 
        
        mom10Data = lookbackInstrumentFeatures.getFeatureDf('mom_10')
        mom30Data = lookbackInstrumentFeatures.getFeatureDf('mom_30')
        ma90Data = lookbackInstrumentFeatures.getFeatureDf('ma_90')
        
        if len(ma90Data.index)>20:
            mom30 = mom30Data.iloc[-1]
            mom10 = mom10Data.iloc[-1]
            ma90 = ma90Data.iloc[-1]
            
            hurst = ma90Data.apply(hurst_f, axis=0)
            
            predictions[(hurst > 0.5) & (mom30 > 0) & (mom10 > 0)] = 1
            predictions[(hurst > 0.5) & (mom30 <= 0) & (mom10 <= 0)] = 0 
            
            predictions[(hurst > 0.5) & (mom30 > 0) & (mom10 <= 0)] = 0.5
            predictions[(hurst > 0.5) & (mom30 <= 0) & (mom10 > 0)] = 0.5
            
            predictions[hurst <= 0.5] = 0.5
            
        else:
            predictions.values[:] = 0.5
        return predictions

    def updateCount(self):
        self.count = self.count + 1

In [5]:
class MyCustomFeature(Feature):

    @classmethod
    def computeForInstrument(cls, updateNum, time, featureParams, featureKey, instrumentManager):
        
        param1Value = featureParams['param1']
        lookbackInstrumentFeatures = instrumentManager.getLookbackInstrumentFeatures()
        lookbackInstrumentValue = lookbackInstrumentFeatures.getFeatureDf('adjClose')
        
        currentValue = lookbackInstrumentValue.iloc[-1]

        if param1Value == 'value1':
            return currentValue * 0.1
        else:
            return currentValue * 0.5
        

In [6]:
tf = MyTradingFunctions()
tsParams = MyTradingParams(tf)
tradingSystem = TradingSystem(tsParams)
results = tradingSystem.startTrading()

Processing data for stock: AAPL
20% done...
40% done...
60% done...
80% done...
Logging all the available market metrics in tensorboard
Logging all the available instrument metrics in tensorboard
2015-01-02 00:00:00
2015-01-05 00:00:00
AAPL
pnl: 0.00
2015-01-06 00:00:00
AAPL
pnl: 0.00
2015-01-07 00:00:00
AAPL
pnl: 0.00
2015-01-08 00:00:00
AAPL
pnl: 0.00
2015-01-09 00:00:00
AAPL
pnl: 0.00
2015-01-12 00:00:00
AAPL
pnl: 0.00
2015-01-13 00:00:00
AAPL
pnl: 0.00
2015-01-14 00:00:00
AAPL
pnl: 0.00
2015-01-15 00:00:00
AAPL
pnl: 0.00
2015-01-16 00:00:00
AAPL
pnl: 0.00
2015-01-19 00:00:00
AAPL
pnl: 0.00
2015-01-20 00:00:00
AAPL
pnl: 0.00
2015-01-21 00:00:00
AAPL
pnl: 0.00
2015-01-22 00:00:00
AAPL
pnl: 0.00
2015-01-23 00:00:00
AAPL
pnl: 0.00
2015-01-26 00:00:00
AAPL
pnl: 0.00
2015-01-27 00:00:00
AAPL
pnl: 0.00
2015-01-28 00:00:00
AAPL
pnl: 0.00
2015-01-29 00:00:00
AAPL
pnl: 0.00
2015-01-30 00:00:00
AAPL
pnl: 0.00
2015-02-02 00:00:00
AAPL
pnl: 0.00
2015-02-03 00:00:00
AAPL
pnl: 0.00
2015-02-04 00:

pnl: -17.55
2015-09-29 00:00:00
AAPL
pnl: -17.55
Position changed to: -1.00
2015-09-30 00:00:00
AAPL
pnl: -14.42
2015-10-01 00:00:00
AAPL
pnl: -15.57
2015-10-02 00:00:00
AAPL
pnl: -14.90
2015-10-05 00:00:00
AAPL
pnl: -15.64
Position changed to: 0.00
2015-10-06 00:00:00
AAPL
pnl: -15.64
2015-10-07 00:00:00
AAPL
pnl: -15.64
2015-10-08 00:00:00
AAPL
pnl: -15.64
2015-10-09 00:00:00
AAPL
pnl: -15.64
Position changed to: -1.00
2015-10-12 00:00:00
AAPL
pnl: -18.07
2015-10-13 00:00:00
AAPL
pnl: -17.59
2015-10-14 00:00:00
AAPL
pnl: -17.77
2015-10-15 00:00:00
AAPL
pnl: -16.30
Position changed to: 0.00
2015-10-16 00:00:00
AAPL
pnl: -16.30
2015-10-19 00:00:00
AAPL
pnl: -16.30
Position changed to: 1.00
2015-10-20 00:00:00
AAPL
pnl: -15.66
2015-10-21 00:00:00
AAPL
pnl: -13.77
Position changed to: 0.00
2015-10-22 00:00:00
AAPL
pnl: -13.77
Position changed to: 1.00
2015-10-23 00:00:00
AAPL
pnl: -12.15
2015-10-26 00:00:00
AAPL
pnl: -8.83
2015-10-27 00:00:00
AAPL
pnl: -12.36
2015-10-28 00:00:00
AAPL
pnl

2016-07-05 00:00:00
AAPL
pnl: 8.97
2016-07-06 00:00:00
AAPL
pnl: 8.12
Position changed to: 0.00
2016-07-07 00:00:00
AAPL
pnl: 8.12
Position changed to: -1.00
2016-07-08 00:00:00
AAPL
pnl: 7.73
2016-07-11 00:00:00
AAPL
pnl: 7.04
Position changed to: 0.00
2016-07-12 00:00:00
AAPL
pnl: 7.04
2016-07-13 00:00:00
AAPL
pnl: 7.04
2016-07-14 00:00:00
AAPL
pnl: 7.04
2016-07-15 00:00:00
AAPL
pnl: 7.04
2016-07-18 00:00:00
AAPL
pnl: 7.04
Position changed to: 1.00
2016-07-19 00:00:00
AAPL
pnl: 8.03
2016-07-20 00:00:00
AAPL
pnl: 8.06
2016-07-21 00:00:00
AAPL
pnl: 8.15
2016-07-22 00:00:00
AAPL
pnl: 7.65
2016-07-25 00:00:00
AAPL
pnl: 6.92
2016-07-26 00:00:00
AAPL
pnl: 5.68
2016-07-27 00:00:00
AAPL
pnl: 5.05
2016-07-28 00:00:00
AAPL
pnl: 10.97
2016-07-29 00:00:00
AAPL
pnl: 12.28
2016-08-01 00:00:00
AAPL
pnl: 12.16
2016-08-02 00:00:00
AAPL
pnl: 13.89
2016-08-03 00:00:00
AAPL
pnl: 12.41
2016-08-04 00:00:00
AAPL
pnl: 13.65
2016-08-05 00:00:00
AAPL
pnl: 14.26
2016-08-08 00:00:00
AAPL
pnl: 15.79
2016-08-09 0

2017-04-21 00:00:00
AAPL
pnl: 35.18
2017-04-24 00:00:00
AAPL
pnl: 35.18
2017-04-25 00:00:00
AAPL
pnl: 35.18
2017-04-26 00:00:00
AAPL
pnl: 35.18
Position changed to: 1.00
2017-04-27 00:00:00
AAPL
pnl: 34.36
2017-04-28 00:00:00
AAPL
pnl: 34.47
2017-05-01 00:00:00
AAPL
pnl: 34.33
2017-05-02 00:00:00
AAPL
pnl: 37.14
2017-05-03 00:00:00
AAPL
pnl: 38.03
2017-05-04 00:00:00
AAPL
pnl: 37.60
2017-05-05 00:00:00
AAPL
pnl: 37.09
2017-05-08 00:00:00
AAPL
pnl: 39.42
2017-05-09 00:00:00
AAPL
pnl: 43.29
2017-05-10 00:00:00
AAPL
pnl: 44.23
2017-05-11 00:00:00
AAPL
pnl: 43.53
2017-05-12 00:00:00
AAPL
pnl: 44.80
2017-05-15 00:00:00
AAPL
pnl: 46.87
2017-05-16 00:00:00
AAPL
pnl: 46.48
2017-05-17 00:00:00
AAPL
pnl: 46.26
2017-05-18 00:00:00
AAPL
pnl: 41.24
2017-05-19 00:00:00
AAPL
pnl: 43.45
2017-05-22 00:00:00
AAPL
pnl: 43.95
2017-05-23 00:00:00
AAPL
pnl: 44.84
2017-05-24 00:00:00
AAPL
pnl: 44.66
2017-05-25 00:00:00
AAPL
pnl: 44.21
2017-05-26 00:00:00
AAPL
pnl: 44.72
Position changed to: 0.00
2017-05-29 0

In [7]:
results

{'instrument_names': ['AAPL'],
 'instrument_stats': [{'pnl': {'AAPL': 0.03297904699999998},
   'score': {'AAPL': -0.21782390972551793}}],
 'pnl': 0.03297904699999998,
 'trading_days': 671,
 'annual_return': 0.01226027180459166,
 'annual_vol': 0.020633536014475658,
 'sharpe_ratio': 0.5941915043543844,
 'score': -0.21782390972551793,
 'capitalUsage': 142.52400554999883,
 'capital': 863.5019917000012,
 'variance': 1.6894555891296192,
 'count_loss': 207,
 'portfolio_value': 1032.979047,
 'total_loss': 227.25633200000004,
 'maxDrawdown': 22.245457999999985,
 'maxPortfolioValue': 1047.308882,
 'count_profit': 221,
 'total_profit': 260.23537899999997}